<a href="https://colab.research.google.com/github/Joselota/Relatorias/blob/main/Colab_SQLite_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Demo: SQLite + Python + Pandas (Colab-ready)

**Objetivo:** tener un notebook listo para clase que muestre cómo:
1) Conectarse a un archivo `.db` de SQLite  
2) Ejecutar consultas `SELECT / INSERT / UPDATE / DELETE` con parámetros  
3) Usar `pandas.read_sql()` para análisis rápido  
4) Exportar resultados a CSV/Excel  
5) (Opcional) Montar Google Drive para guardar/leer archivos

> Nota: Este notebook está pensado para usarse en **Google Colab** con el archivo `mi_base.db`.


In [1]:

# (Opcional) Instalar soporte para exportar a Excel
# En Colab normalmente ya está, pero por si acaso:
try:
    import openpyxl  # noqa: F401
except ImportError:
    !pip -q install openpyxl



## 1) Traer el archivo `mi_base.db` a Colab

Elige **una** de estas dos opciones:
- **A. Subir el archivo** desde tu computador (rápido para demo).
- **B. Montar Drive** y usar un `.db` guardado en tu Google Drive.


In [2]:

# A) Subir archivo desde tu computador (recomendado para la demo)
# Ejecuta esta celda y selecciona el archivo mi_base.db
try:
    from google.colab import files
    up = files.upload()  # aparecerá el diálogo para subir
except Exception as e:
    print("No estás en Colab o no se pudo abrir el diálogo de carga:", e)


Saving mi_base.db to mi_base.db


In [4]:

# B) (Opcional) Montar Google Drive y apuntar a una ruta .db
# Descomenta y edita el path si prefieres usar tu Drive.
# from google.colab import drive
# drive.mount('/content/drive')
# db_path = '/content/drive/MyDrive/ruta/a/mi_base.db'
# !ls -la /content/drive/MyDrive | head -n 20



## 2) Conexión a SQLite
Si subiste el archivo en la opción A, el nombre del archivo debe ser `mi_base.db` en el directorio actual.


In [5]:

import sqlite3, pandas as pd, os

# Detecta si existe mi_base.db en el directorio actual; si no, intenta usar db_path (Drive)
db_file = 'mi_base.db' if os.path.exists('mi_base.db') else globals().get('db_path', 'mi_base.db')

conn = sqlite3.connect(db_file)
cursor = conn.cursor()
print("Conectado a:", db_file)


Conectado a: mi_base.db



## 3) Listar tablas y hacer un `SELECT` sencillo


In [6]:

# Listar tablas
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", conn)
tables


,name
0,departamentos
1,empleados
2,sqlite_sequence


In [7]:

# SELECT básico
df_empleados = pd.read_sql("SELECT * FROM empleados;", conn)
df_empleados.head()


,id,nombre,edad,departamento,salario
0,1,Ana,28,Ventas,1200.50
1,2,Luis,34,TI,2500.00
2,3,María,41,Recursos Humanos,1800.75
3,4,Pedro,25,Ventas,1100.00
4,5,Lucía,30,TI,2700.25



## 4) CRUD básico con parámetros (mejor práctica contra inyección SQL)


In [8]:

# INSERT con parámetros
nuevo = ('Carla', 29, 'Ventas', 1350.0)
cursor.execute(
    "INSERT INTO empleados (nombre, edad, departamento, salario) VALUES (?, ?, ?, ?);",
    nuevo
)
conn.commit()

pd.read_sql("SELECT COUNT(*) AS n_empleados FROM empleados;", conn)


,n_empleados
0,6


In [9]:

# UPDATE con parámetros: subir salario de Carla en 5%
cursor.execute(
    "UPDATE empleados SET salario = salario * 1.05 WHERE nombre = ?;",
    ('Carla',)
)
conn.commit()

pd.read_sql("SELECT nombre, salario FROM empleados WHERE nombre='Carla';", conn)


,nombre,salario
0,Carla,1417.5


In [10]:

# DELETE con parámetros: eliminar a Carla (solo como ejemplo de demo)
cursor.execute(
    "DELETE FROM empleados WHERE nombre = ?;",
    ('Carla',)
)
conn.commit()

pd.read_sql("SELECT COUNT(*) AS n_empleados FROM empleados;", conn)


,n_empleados
0,5



## 5) Agregaciones y JOINs con `pandas.read_sql()`


In [11]:

# Promedio salarial por departamento
query_avg = '''
SELECT departamento, ROUND(AVG(salario),2) AS salario_promedio, COUNT(*) AS n
FROM empleados
GROUP BY 1
ORDER BY 2 DESC;
'''
pd.read_sql(query_avg, conn)


,departamento,salario_promedio,n
0,TI,2600.13,2
1,Recursos Humanos,1800.75,1
2,Ventas,1150.25,2


In [12]:

# JOIN empleados ↔ departamentos (por nombre de departamento)
query_join = '''
SELECT e.nombre, e.departamento, d.ubicacion, e.salario
FROM empleados e
LEFT JOIN departamentos d ON d.nombre = e.departamento
ORDER BY e.departamento, e.nombre;
'''
pd.read_sql(query_join, conn).head(10)


,nombre,departamento,ubicacion,salario
0,María,Recursos Humanos,Concepción,1800.75
1,Lucía,TI,Valparaíso,2700.25
2,Luis,TI,Valparaíso,2500.00
3,Ana,Ventas,Santiago,1200.50
4,Pedro,Ventas,Santiago,1100.00



## 6) Exportar resultados a CSV / Excel


In [13]:

df_export = pd.read_sql(query_join, conn)
df_export.to_csv('empleados_departamentos.csv', index=False)
df_export.to_excel('empleados_departamentos.xlsx', index=False)
print("Archivos generados: empleados_departamentos.csv y empleados_departamentos.xlsx")


Archivos generados: empleados_departamentos.csv y empleados_departamentos.xlsx



## 7) Cerrar conexión


In [15]:

conn.close()
print("Conexión cerrada.")


Conexión cerrada.
